# Classify WT and Null Genotypes Logistic Regression
Plates 3, 3p, and 5 are used in all splits to classify genotypes either (WT or Null)
The feature selected data is used in all data splits.
Pre-evaluation metrics are stored from all splits and these plates.

In [1]:
import pathlib
import random
import warnings
from collections import defaultdict

import numpy as np
import pandas as pd
from joblib import dump
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import parallel_backend

In [2]:
MODEL_ID = "logreg"
ROLE = "train"

# ---- Standard logging setup for this project ----

import logging
from datetime import datetime
import pathlib

# ============================================
# 1) Choose a RUN_ID
# ============================================
RUN_ID = datetime.now().strftime("%m_%d_%H_%M")




RUN_ID = "12_08_16_45"
ANALYSIS_TYPE = "train"


def setup_logger(
    run_id: str,
    model_id: str,
    role: str,
    log_dir: str = "logs",
    analysis_type: str = ANALYSIS_TYPE,
) -> logging.Logger:
    """
    Create a logger that writes to both stdout and a log file.

    - Logger name:  "<analysis_type>_<run_id>_<model_id>_<role>"
    - Log file:     "log_<analysis_type>_<run_id>_<model_id>.log" in `log_dir`
      (shared by all notebooks for the same model & run & analysis_type).
    """
    log_path = pathlib.Path(log_dir)
    log_path.mkdir(exist_ok=True)

    logger_name = f"{analysis_type}_{run_id}_{model_id}_{role}"
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    logger.propagate = False  # don't duplicate logs to root logger

    # Avoid adding handlers multiple times if the cell is re-run
    if not logger.handlers:
        # Common formatter for both handlers
        formatter = logging.Formatter(
            fmt="%(asctime)s [%(name)s] %(levelname)s: %(message)s",
            datefmt="%Y-%m-%dT%H:%M:%S",
        )

        # Stream handler (stdout)
        stream_handler = logging.StreamHandler()
        stream_handler.setLevel(logging.INFO)
        stream_handler.setFormatter(formatter)
        logger.addHandler(stream_handler)

        # File handler (one file per analysis_type + run_id + model_id)
        log_file = log_path / f"log_{analysis_type}_{run_id}_{model_id}.log"
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    return logger



logger = setup_logger(RUN_ID, MODEL_ID, ROLE)
logger.info("Initialized logger.")


2025-12-06T16:47:13 [train_12_08_16_45_logreg_train] INFO: Initialized logger.


## Find the root of the git repo on the host system

In [3]:
# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

## Define paths

### Input

In [4]:
# OPTIONAL: If the data (within the cell painting directory) is stored in a different location, add location here
repo_dir = pathlib.Path(
    "/Users/marktalbot/Documents/VC Studio Homework Folders/HighRisk/nf1_schwann_cell_painting_data"
)

# Set data level
data_level = "cleaned"

# Main directory path (converted or cleaned data)
if data_level == "cleaned":
    data_dir = pathlib.Path(
        repo_dir / "3.processing_features/data/single_cell_profiles/cleaned_sc_profiles"
    )
else:
    data_dir = pathlib.Path(
        repo_dir / "3.processing_features/data/single_cell_profiles"
    )

plate3df_path = pathlib.Path(data_dir / "Plate_3_sc_feature_selected.parquet").resolve(
    strict=True
)
plate3pdf_path = pathlib.Path(
    data_dir / "Plate_3_prime_sc_feature_selected.parquet"
).resolve(strict=True)
plate5df_path = pathlib.Path(data_dir / "Plate_5_sc_feature_selected.parquet").resolve(
    strict=True
)

plate3df = pd.read_parquet(plate3df_path)
plate3pdf = pd.read_parquet(plate3pdf_path)
plate5df = pd.read_parquet(plate5df_path)

logger.info("Number of single-cells total per plate:")
logger.info(f"Plate 3: {plate3df.shape[0]}")
logger.info(f"Plate 3 prime: {plate3pdf.shape[0]}")
logger.info(f"Plate 5: {plate5df.shape[0]}")

# Set the seed
rng = np.random.default_rng(0)


2025-12-06T16:47:14 [train_12_08_16_45_logreg_train] INFO: Number of single-cells total per plate:
2025-12-06T16:47:14 [train_12_08_16_45_logreg_train] INFO: Plate 3: 10206
2025-12-06T16:47:14 [train_12_08_16_45_logreg_train] INFO: Plate 3 prime: 5126
2025-12-06T16:47:14 [train_12_08_16_45_logreg_train] INFO: Plate 5: 5348


### Outputs

In [5]:
data_path = pathlib.Path("data")
data_path.mkdir(parents=True, exist_ok=True)

## Splitting and Processing
Functions to split and process data

In [6]:
gene_column = "Metadata_genotype"


def down_sample_by_genotype(_df):
    """
    Return an equal number of cells from each genotype.
    The number of cells in a genotype is the minimum number of cells from all genotypes.

    Parameters
    ----------
    _df: Pandas Dataframe
        The data to be downsampled by the gene_column column.

    Returns
    -------
    The dataframe down-sampled by genotype.
    """

    min_gene = _df[gene_column].value_counts().min()
    return _df.groupby(gene_column, group_keys=False).apply(
        lambda x: x.sample(n=min_gene, random_state=0)
    )


def process_plates(_df):
    """
    Drop rows with nans from the single cell data and remove HET cells.

    Parameters
    ----------
    _df: Pandas Dataframe
        Uncleaned plate data with nans and HET cells to be removed. Contains the column "Metadata_genotype".

    Returns
    -------
    _df: Pandas Dataframe
        Cleaned single cell data by removing nans and HET cells.
    """

    _df.dropna(inplace=True)
    _df = _df.loc[_df[gene_column] != "HET"]
    return _df


def shuffle_data(_X):
    """
    Shuffle the columns of the input dataframe independently.

    Parameters
    ----------
    _X: Pandas Dataframe
        Input feature data for shuffling the columns.
    """

    for column in _X.columns:
        _X[column] = rng.permutation(_X[column])


def store_pre_evaluation_data(_X, _y, _metadata, _datasplit):
    """
    Store model data to evaluate performance.

    Parameters
    ----------
    _X: Pandas Dataframe
        Feature dataframe from a given plate and data split.

    _y: Numpy Array
        A numerically-encoded label vector ordered according to _X.

    _metadata: Pandas Dataframe
        Plate name.

    _datasplit: String
        Data split name.
    """

    eval_data[f"probability_{probability_class}"].extend(
        logreg.predict_proba(_X)[:, 1].tolist()
    )
    eval_data["datasplit"].extend([_datasplit] * _X.shape[0])
    eval_data["predicted_genotype"].extend(logreg.predict(_X).tolist())
    eval_data["true_genotype"].extend(_y.tolist())
    for meta_col in _metadata.columns:
        eval_data[meta_col].extend(_metadata[meta_col].tolist())

## Split and process plates

In [7]:
def create_splits(_wells, _plate):
    """
    Create data splits for model training. The splits are rest (train and validation) and test.

    Parameters
    ----------
    _wells: List(String)
        The well names from which single cells will be used in the test set.

    _plate: Pandas Dataframe
        Single cell data from one of the plate's containing a "Metadata_Well" column.

    Returns
    -------
    Dataframes of the split single cell data.
    """

    return (
        _plate[~_plate["Metadata_Well"].isin(_wells)],
        _plate[_plate["Metadata_Well"].isin(_wells)],
    )

In [8]:
plate3df = process_plates(plate3df)
p3_wells = ["C11", "E11", "C3", "F3"]
rest3df, test3df = create_splits(p3_wells, plate3df)
rest3df, test3df = down_sample_by_genotype(rest3df), down_sample_by_genotype(test3df)

plate3pdf = process_plates(plate3pdf)
p3p_wells = ["F11", "G11", "C3", "F3"]
rest3pdf, test3pdf = create_splits(p3p_wells, plate3pdf)
rest3pdf, test3pdf = down_sample_by_genotype(rest3pdf), down_sample_by_genotype(
    test3pdf
)

plate5df = process_plates(plate5df)
p5_wells = ["C9", "E11", "E3", "G3"]
rest5df, test5df = create_splits(p5_wells, plate5df)
rest5df, test5df = down_sample_by_genotype(rest5df), down_sample_by_genotype(test5df)

## Combine plate columns across each data split

In [9]:
# Columns common to all plates
plate_cols = list(
    set(plate5df.columns) & set(plate3df.columns) & set(plate3pdf.columns)
)

restdf = pd.concat(
    [rest3df[plate_cols], rest3pdf[plate_cols], rest5df[plate_cols]], ignore_index=True
).reset_index(drop=True)

testdf = pd.concat(
    [test3df[plate_cols], test3pdf[plate_cols], test5df[plate_cols]], ignore_index=True
).reset_index(drop=True)

## Encode genotypes and extract feature data

In [10]:
meta_cols = testdf.filter(like="Metadata").columns
feat_cols = testdf.drop(columns=meta_cols).columns

In [11]:
le = LabelEncoder()

y = le.fit_transform(restdf["Metadata_genotype"])
X = restdf.drop(columns=meta_cols)

y_test = le.fit_transform(testdf["Metadata_genotype"])
X_test = testdf.drop(columns=meta_cols)

# Class for saving probabilities
probability_class = le.inverse_transform([1])[0]

# Train Models

## Specify parameters for training

In [ ]:
logreg_params = {
    "max_iter": 250,
    "random_state": 0,
    "n_jobs": -1,
    "penalty": "l2",
}

# Random sampling range of hyperparameter (same search space as before)
param_ranges = {
    "C": (0.0, 200.0)
}

# Target *total* number of trials in the Optuna study
rand_iter = 500

# Best accuracy and best hyperparameters (will be set from Optuna)
best_acc = 0.0
best_hp = None

# Number of folds
n_splits = 8


## Hyperparameter search

In [13]:
from collections import defaultdict
import os
import optuna
from optuna.trial import TrialState

# Make sure the directory for the SQLite file exists
os.makedirs("tuning", exist_ok=True)

# Store model results for evaluation (kept as in your original code)
eval_data = defaultdict(list)
best_hparam = None

# Create/load persistent Optuna study
study = optuna.create_study(
    study_name="logreg_cv",
    direction="maximize",
    storage="sqlite:///tuning/optuna_logreg.sqlite3",  # <-- changed here
    load_if_exists=True,
)


# How many trials have already successfully finished?
completed_trials = [t for t in study.trials if t.state == TrialState.COMPLETE]
n_complete = len(completed_trials)

# Initialize best from existing study, if any
if n_complete > 0:
    best_trial_so_far = study.best_trial
    best_acc = float(best_trial_so_far.value)
    best_hp = best_trial_so_far.params
else:
    best_acc = 0.0
    best_hp = None

# Decide how many *new* trials to run this time
n_to_run = max(0, rand_iter - n_complete)

logger.info(
    f"[INFO] Loaded Optuna study 'logreg_cv' with "
    f"{len(study.trials)} total trials ({n_complete} completed). "
    f"Target total trials = {rand_iter}. "
    f"Running {n_to_run} new trials in this session."
)

# Run only the additional trials needed to reach rand_iter total completed trials
for i in range(n_to_run):
    trial = study.ask()

    # Same hyperparameter range as your original random search
    C_low, C_high = param_ranges["C"]
    rparams = {
        "C": trial.suggest_float("C", C_low, C_high),
    }

    # Global index for logging (so you see progression across reruns)
    global_idx = n_complete + i  # 0-based index among desired trials

    logger.info(
        f"[INFO] Iteration {global_idx + 1} of {rand_iter}: "
        f"trying C = {rparams.get('C', float('nan')):.3f} "
        f"(Optuna trial {trial.number})"
    )

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    # Combine parameters in current search with logistic regression parameters
    comb_params = logreg_params | rparams

    # Cumulative accuracy for all folds (reset each trial)
    acc = 0.0

    # Loop through the folds
    for fold, (train_index, val_index) in enumerate(skf.split(X, y)):

        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y[train_index], y[val_index]

        X_val_shuf = X_val.copy()
        shuffle_data(X_val_shuf)

        # Prevent the convergence warning in sklearn
        with parallel_backend("multiprocessing"):
            with warnings.catch_warnings():
                warnings.filterwarnings(
                    "ignore", category=ConvergenceWarning, module="sklearn"
                )
                logreg = LogisticRegression(**comb_params)
                logreg.fit(X_train, y_train)

        preds = logreg.predict(X_val)
        preds_shuf = logreg.predict(X_val_shuf)
        acc += accuracy_score(y_val, preds)

        store_pre_evaluation_data(
            X_val,
            y_val,
            restdf.iloc[val_index][meta_cols],
            "val",
        )
        store_pre_evaluation_data(
            X_val_shuf,
            y_val,
            restdf.iloc[val_index][meta_cols],
            "shuffled_val",
        )

    # Average accuracy for the folds
    acc = acc / n_splits

    # Track best performance within this run as before
    if acc > best_acc:
        best_hparam = eval_data.copy()
        best_acc = acc
        best_hp = rparams

    # Tell Optuna about this trial's result
    study.tell(trial, acc)

    logger.info(
        f"[INFO] Completed Optuna trial {trial.number}: "
        f"mean CV accuracy = {acc:.4f}"
    )

# After running any new trials (or if n_to_run == 0), sync to *global* best over the study
completed_trials = [t for t in study.trials if t.state == TrialState.COMPLETE]
if not completed_trials:
    raise RuntimeError("No completed Optuna trials in the study. Nothing to select as best.")

best_trial = study.best_trial
best_acc = float(best_trial.value)
best_hp = best_trial.params

logger.info(f"Best average validation accuracy (all completed trials) = {best_acc:.4f}")
logger.info(f"Best hyperparameters: {best_hp}")



/opt/miniconda3/envs/nf1_analysis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-06 16:47:14,966] Using an existing study with name 'logreg_cv' instead of creating a new one.
2025-12-06T16:47:14 [train_12_08_16_45_logreg_train] INFO: [INFO] Loaded Optuna study 'logreg_cv' with 19 total trials (18 completed). Target total trials = 50. Running 32 new trials in this session.
2025-12-06T16:47:14 [train_12_08_16_45_logreg_train] INFO: [INFO] Iteration 19 of 50: trying C = 107.379 (Optuna trial 19)
/opt/miniconda3/envs/nf1_analysis/lib/python3.9/site-packages/sklearn/utils/deprecation.py:71: FutureWarning: Class parallel_backend is deprecated; deprecated in 1.5 to be removed in 1.7. Use joblib.{} instead.
  warnings.warn(msg, category=FutureWarning)
/opt/miniconda3/envs/nf1_analysis/lib/python3.9/s

## Retrain model

In [14]:
logreg_params = {
    "max_iter": 3000,
    "random_state": 0,
    "n_jobs": -1,
    "penalty": "l2",
}

comb_params = logreg_params | best_hp

logreg = LogisticRegression(**comb_params)
logreg.fit(X, y)

LogisticRegression(C=71.05932179251381, max_iter=3000, n_jobs=-1,
                   random_state=0)

## Shuffle train and validation data

In [15]:
X_shuf = X.copy()
shuffle_data(X_shuf)

X_test_shuf = X_test.copy()
shuffle_data(X_test_shuf)

# Save models and model data

## Store pre-evaluation split data

In [16]:
store_pre_evaluation_data(X, y, restdf[meta_cols], "train")
store_pre_evaluation_data(X_shuf, y, restdf[meta_cols], "shuffled_train")

store_pre_evaluation_data(X_test, y_test, testdf[meta_cols], "test")
store_pre_evaluation_data(X_test_shuf, y_test, testdf[meta_cols], "shuffled_test")

In [17]:
suffix = "_qc" if data_level == "cleaned" else ""

dump(logreg, f"{data_path}/trained_nf1_model{suffix}.joblib")
dump(le, f"{data_path}/trained_nf1_model_label_encoder{suffix}.joblib")
pd.DataFrame(eval_data).to_parquet(f"{data_path}/nf1_model_pre_evaluation_results{suffix}.parquet")